__Probing Language Models__

This notebook serves as a start for your NLP2 assignment on probing Language Models. This notebook will become part of the contents that you will submit at the end, so make sure to keep your code (somewhat) clean :-)

__note__: This assignment is not dependent on big fancy GPUs. I run all this stuff on my own 3 year old CPU, without any Colab hassle. So it's up to you to decide how you want to run it.

# Models

For the Transformer models you are advised to make use of the `transformers` library of Huggingface: https://github.com/huggingface/transformers
Their library is well documented, and they provide great tools to easily load in pre-trained models.

In [1]:
#
## Your code for initializing the transformer model(s)
#
# Note that transformer models use their own `tokenizer`, that should be loaded in as well.
#
from transformers import *

tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2')
model = GPT2LMHeadModel.from_pretrained('distilgpt2')

c:\Users\laure\miniconda3\lib\site-packages\transformers\generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(
c:\Users\laure\miniconda3\lib\site-packages\torchaudio\backend\utils.py:74: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")
loading file vocab.json from cache at C:\Users\laure/.cache\huggingface\hub\models--distilgpt2\snapshots\38cc92ec43315abd5136313225e95acc5986876c\vocab.json
loading file merges.txt from cache at C:\Users\laure/.cache\huggingface\hub\models--distilgpt2\snapshots\38cc92ec43315abd5136313225e95acc5986876c\merges.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at None
loading configuration file config.json from cache at C:

In [2]:
#
## Your code for initializing the rnn model(s)
#
# The Gulordava LSTM model can be found here: 
# https://drive.google.com/file/d/19Lp3AM4NEPycp_IBgoHfLc_V456pmUom/view?usp=sharing
# You can read more about this model in the original paper here: https://arxiv.org/pdf/1803.11138.pdf
#
# N.B: I have altered the RNNModel code to only output the hidden states that you are interested in.
# If you want to do more experiments with this model you could have a look at the original code here:
# https://github.com/facebookresearch/colorlessgreenRNNs/blob/master/src/language_models/model.py
#
from collections import defaultdict
from lstm.model import RNNModel
import torch


model_location = 'state_dict.pt'  # <- point this to the location of the Gulordava .pt file
lstm = RNNModel('LSTM', 50001, 650, 650, 2)
lstm.load_state_dict(torch.load(model_location))


# This LSTM does not use a Tokenizer like the Transformers, but a Vocab dictionary that maps a token to an id.
with open('lstm/vocab.txt', encoding ="utf8") as f:
    w2i = {w.strip(): i for i, w in enumerate(f)}

# vocab = {}
vocab = defaultdict(lambda: w2i["<unk>"])
vocab.update(w2i)
# vocab.default_factory = lambda: w2i["<unk>"]

It is a good idea that before you move on, you try to feed some text to your LMs; and check if everything works accordingly. 

# Data

For this assignment you will train your probes on __treebank__ corpora. A treebank is a corpus that has been *parsed*, and stored in a representation that allows the parse tree to be recovered. Next to a parse tree, treebanks also often contain information about part-of-speech tags, which is exactly what we are after now.

The treebank you will use for now is part of the Universal Dependencies project. I provide a sample of this treebank as well, so you can test your setup on that before moving on to larger amounts of data.

Make sure you accustom yourself to the format that is created by the `conllu` library that parses the treebank files before moving on. For example, make sure you understand how you can access the pos tag of a token, or how to cope with the tree structure that is formed using the `to_tree()` functionality.

In [3]:
# READ DATA
from typing import List
from conllu import parse_incr, TokenList


# If stuff like `: str` and `-> ..` seems scary, fear not! 
# These are type hints that help you to understand what kind of argument and output is expected.
def parse_corpus(filename: str) -> List[TokenList]:
    data_file = open(filename, encoding="utf-8")

    ud_parses = list(parse_incr(data_file))
    
    return ud_parses


ud_parses = parse_corpus('data/sample/en_ewt-ud-train.conllu')

# Generating Representations

We now have our data all set, our models are running and we are good to go!

The next step is now to create the model representations for the sentences in our corpora. Once we have generated these representations we can store them, and train additional diagnostic (/probing) classifiers on top of the representations.

There are a few things you should keep in mind here. Read these carefully, as these tips will save you a lot of time in your implementation.
1. Transformer models make use of Byte-Pair Encodings (BPE), that chunk up a piece of next in subword pieces. For example, a word such as "largely" could be chunked up into "large" and "ly". We are interested in probing linguistic information on the __word__-level. Therefore, we will follow the suggestion of Hewitt et al. (2019a, footnote 4), and create the representation of a word by averaging over the representations of its subwords. So the representation of "largely" becomes the average of that of "large" and "ly".


2. Subword chunks never overlap multiple tokens. In other words, say we have a phrase like "None of the", then the tokenizer might chunk that into "No"+"ne"+" of"+" the", but __not__ into "No"+"ne o"+"f the", as those chunks overlap multiple tokens. This is great for our setup! Otherwise it would have been quite challenging to distribute the representation of a subword over the 2 tokens it belongs to.


3. **Important**: If you closely examine the provided treebank, you will notice that some tokens are split up into multiple pieces, that each have their own POS-tag. For example, in the first sentence the word "Al-Zaman" is split into "Al", "-", and "Zaman". In such cases, the conllu `TokenList` format will add the following attribute: `('misc', OrderedDict([('SpaceAfter', 'No')]))` to these tokens. Your model's tokenizer does not need to adhere to the same tokenization. E.g., "Al-Zaman" could be split into "Al-"+"Za"+"man", making it hard to match the representations with their correct pos-tag. Therefore I recommend you to not tokenize your entire sentence at once, but to do this based on the chunking of the treebank. <br /><br />
Make sure to still incoporate the spaces in a sentence though, as these are part of the BPE of the tokenizer. That is, the tokenizer uses a different token id for `"man"`, than it does for `" man"`: the former could be part of `" woman"`=`" wo`"+`"man"`, whereas the latter would be the used in case *man* occurs at the start of a word. The tokenizer for GPT-2 adds spaces at the start of a token (represented as a `Ġ` symbol). This means that you should keep track whether the previous token had the `SpaceAfter` attribute set to `'No'`: in case it did not, you should manually prepend a `" "` ahead of the token.


4. The LSTM LM does not have the issues related to subwords, but is far more restricted in its vocabulary. Make sure you keep the above points in mind though, when creating the LSTM representations. You might want to write separate functions for the LSTM, but that is up to you.


5. The huggingface transformer models don't return the hidden state by default. To achieve this you can pass `output_hidden_states=True` to a model forward pass. The hidden states are then returned for all intermediate layers as well, the latest entry in this list corresponds to the top layer.


6. **N.B.**: Make sure that when you run a sentence through your model, you do so within a `with torch.no_grad():` block, and that you have run `model.eval()` beforehand as well (to disable dropout).


7. **N.B.**: Make sure to use a token's ``["form"]`` attribute, and not the ``["lemma"]``, as the latter will stem any relevant morphological information from the token. We don't want this, because we want to feed well-formed, grammatical sentences to our model.


I would like to stress that if you feel hindered in any way by the simple code structure that is presented here, you are free to modify it :-) Just make sure it is clear to an outsider what you're doing, some helpful comments never hurt.

In [4]:
# FETCH SENTENCE REPRESENTATIONS
from torch import Tensor
import pickle



# Should return a tensor of shape (num_tokens_in_corpus, representation_size)
# Make sure you correctly average the subword representations that belong to 1 token!

def fetch_sen_reps(ud_parses: List[TokenList], model, tokenizer, concat=True) -> Tensor:
    model.eval()


    if model.__class__.__name__ == 'RNNModel':
        print("debug fetch_sen_reps LSTM, concat=", concat)
        vocab = tokenizer
        outputs = []

        with torch.no_grad():
            for sent in ud_parses:
                hidden = model.init_hidden(1)
                word_ids = [vocab[word['form'].replace(' ', '')] for word in sent]
                try:
                    curr_out = model(torch.tensor([word_ids]), hidden)
                except:
                    print([word['form'].replace(' ', '') for word in sent])
                    print(word_ids)
                outputs.append(curr_out.squeeze())
        
        if concat:
            return torch.cat(outputs, dim=0).squeeze()
        else:
#             print([len(x) for x in outputs])
#             print(outputs[0].shape)
            here_out = torch.nn.utils.rnn.pad_sequence(outputs, batch_first=True)
            try:
                return here_out #outputs#.squeeze()
            except:
                print([len(x) for x in outputs])
                print(here_out.shape)
#         return torch.cat(outputs, dim=0).squeeze()

    if model.__class__.__name__ == 'GPT2LMHeadModel':
        print("debug fetch_sen_reps GPT2, concat=", concat)
        with torch.no_grad():
            SpaceAfter = False
            
            outputs = []
            for sent in ud_parses:
                tokens = []
                for word in sent:
                    if SpaceAfter:
                        word['form'] = ' ' + word['form']
                    if word['misc'] == {'SpaceAfter': 'No'}:
                        SpaceAfter = False
                    else:
                        SpaceAfter = True
                
                    tokenized_text = tokenizer(word['form'])['input_ids']
                    tokens.extend(tokenized_text)
                    word['len'] = len(tokenized_text)

                model_outputs = model(torch.tensor(tokens).unsqueeze(0), output_hidden_states=True)
                last_hidden_state = model_outputs.hidden_states[-1].squeeze(0)

                idx = 0
                word_reps = []
                for word in sent:
                    len_word = word['len']
                    word_hidden_states = torch.mean(last_hidden_state[idx :idx + len_word], dim=0)
                    word_reps.append(word_hidden_states)
                    idx += len_word

                word_reps = torch.stack(word_reps)
                outputs.append(word_reps)
                    
            
            if concat:
                return torch.cat(outputs, dim=0)
            else:
                return torch.nn.utils.rnn.pad_sequence(outputs, batch_first=True)  #outputs
#             return torch.cat(outputs, dim=0)
                    

      

    

To validate your activation extraction procedure I have set up the following assertion function as a sanity check. It compares your representation of the first sentence in the corpus against a pickled version of mine. 

For this I used `distilgpt2`.

In [5]:
def error_msg(model_name, gold_embs, embs, i2w):
    with open(f'{model_name}_tokens1.pickle', 'rb') as f:
        sen_tokens = pickle.load(f)
        
    diff = torch.abs(embs - gold_embs)
    max_diff = torch.max(diff)
    avg_diff = torch.mean(diff)
    
    print(f"{model_name} embeddings don't match!")
    print(f"Max diff.: {max_diff:.4f}\nMean diff. {avg_diff:.4f}")

    print("\nCheck if your tokenization matches with the original tokenization:")
    for idx in sen_tokens.squeeze():
        if isinstance(i2w, list):
            token = i2w[idx]
        else:
            token = i2w.convert_ids_to_tokens(idx.item())
        print(f"{idx:<6} {token}")


def assert_sen_reps(model, tokenizer, lstm, vocab):
    with open('distilgpt2_emb1.pickle', 'rb') as f:
        distilgpt2_emb1 = pickle.load(f)
        
    with open('lstm_emb1.pickle', 'rb') as f:
        lstm_emb1 = pickle.load(f)
    
    corpus = parse_corpus('data/sample/en_ewt-ud-train.conllu')[:1]
    
    own_distilgpt2_emb1 = fetch_sen_reps(corpus, model, tokenizer)
    own_lstm_emb1 = fetch_sen_reps(corpus, lstm, vocab)
    
    assert distilgpt2_emb1.shape == own_distilgpt2_emb1.shape, \
        f"Distilgpt2 shape mismatch: {distilgpt2_emb1.shape} (gold) vs. {own_distilgpt2_emb1.shape} (yours)"
    assert lstm_emb1.shape == own_lstm_emb1.shape, \
        f"LSTM shape mismatch: {lstm_emb1.shape} (gold) vs. {own_lstm_emb1.shape} (yours)"

    if not torch.allclose(distilgpt2_emb1, own_distilgpt2_emb1, rtol=1e-3, atol=1e-3):
        error_msg("distilgpt2", distilgpt2_emb1, own_distilgpt2_emb1, tokenizer)
    if not torch.allclose(lstm_emb1, own_lstm_emb1, rtol=1e-3, atol=1e-3):
        error_msg("lstm", lstm_emb1, own_lstm_emb1, list(vocab.keys()))


assert_sen_reps(model, tokenizer, lstm, vocab)

debug fetch_sen_reps GPT2, concat= True
debug fetch_sen_reps LSTM, concat= True


Next, we should define a function that extracts the corresponding POS labels for each activation, which we do based on the **``"upostag"``** attribute of a token (so not the ``xpostag`` attribute). These labels will be transformed to a tensor containing the label index for each item.

In [6]:
# FETCH POS LABELS
from typing import Optional, Dict, Tuple

# Should return a tensor of shape (num_tokens_in_corpus,)
# Make sure that when fetching these pos tags for your train/dev/test corpora you share the label vocabulary.
def fetch_pos_tags(ud_parses: List[TokenList], pos_vocab: Optional[Dict[str, int]] = None) -> Tuple[Tensor, Dict[str, int]]:
    pos_tags = []

    if pos_vocab is None:
        pos_vocab = {}
    for sent in ud_parses:
        for token in sent:  
            if token['upostag'] not in pos_vocab.keys():
                pos_vocab[token['upostag']] = len(pos_vocab)
            pos_tags.append(pos_vocab[token['upostag']])
             
    return torch.tensor(pos_tags), pos_vocab
    

# fetch_pos_tags(parse_corpus('data/sample/en_ewt-ud-train.conllu'))
fetch_pos_tags(ud_parses)

(tensor([0, 1, 0,  ..., 3, 0, 4]),
 {'PROPN': 0,
  'PUNCT': 1,
  'ADJ': 2,
  'NOUN': 3,
  'VERB': 4,
  'DET': 5,
  'ADP': 6,
  'AUX': 7,
  'PRON': 8,
  'PART': 9,
  'SCONJ': 10,
  'NUM': 11,
  'ADV': 12,
  'CCONJ': 13})

Finally, we can combine all these methods to retrieve the representations (`fetch_sen_reps`) and the corresponding labels (`fetch_pos_tags`). If you are still debugging and testing your setup you can set the `use_sample` variable to `True`, and once everything works you can extract the full corpus by setting it to `False`.

The reason we pass the `train_vocab` to the data creation of the `dev` and `test` data is that we want to use the same label vocabulary across the different train/dev/test splits.

In [7]:
import os

# Function that combines the previous functions, and creates 2 tensors for a .conllu file: 
# 1 containing the token representations, and 1 containing the (tokenized) pos_tags.

def create_data(ud_parses, filename: str, lm, w2i, pos_vocab=None):
    ud_parses = parse_corpus(filename)
    
    sen_reps = fetch_sen_reps(ud_parses, lm, w2i, concat=True)
    pos_tags, pos_vocab = fetch_pos_tags(ud_parses, pos_vocab=pos_vocab)
    
    return sen_reps, pos_tags, pos_vocab


lm = model  # or `lstm`
w2i = tokenizer  # or `vocab`
# lm = lstm  # or `lstm`
# w2i = vocab  # or `vocab`

use_sample = False

train_x, train_y, train_vocab = create_data(
    ud_parses,
    os.path.join('data', 'sample' if use_sample else '', 'en_ewt-ud-train.conllu'),
    lm, 
    w2i
)

dev_x, dev_y, _ = create_data(
    ud_parses,
    os.path.join('data', 'sample' if use_sample else '', 'en_ewt-ud-dev.conllu'),
    lm, 
    w2i,
    pos_vocab=train_vocab
)

test_x, test_y, _ = create_data(
    ud_parses,
    os.path.join('data', 'sample' if use_sample else '', 'en_ewt-ud-test.conllu'),
    lm,
    w2i,
    pos_vocab=train_vocab
)

debug fetch_sen_reps GPT2, concat= True
debug fetch_sen_reps GPT2, concat= True
debug fetch_sen_reps GPT2, concat= True


In [8]:
print(len(test_x), len(test_y))

1153 1153


# Diagnostic Classification

We now have our models, our data, _and_ our representations all set! Hurray, well done. We can finally move onto the cool stuff, i.e. training the diagnostic classifiers (DCs).

DCs are simple in their complexity on purpose. To read more about why this is the case you could already have a look at the "Designing and Interpreting Probes with Control Tasks" by Hewitt and Liang (esp. Sec. 3.2).

A simple linear classifier will suffice for now, don't bother with adding fancy non-linearities to it.

I am personally a fan of the `skorch` library, that provides `sklearn`-like functionalities for training `torch` models, but you are free to train your dc using whatever method you prefer.

As this is an Artificial Intelligence master and you have all done ML1 + DL, I expect you to use your train/dev/test splits correctly ;-)

In [16]:
# DIAGNOSTIC CLASSIFIER
# !python -m pip install -U skorch
import os


from torch import nn
from skorch import NeuralNetClassifier

class ClassifierModule(nn.Module):
    def __init__(self, x_dim, y_dim):
        super().__init__()
        self.classifier = nn.Linear(x_dim, y_dim)
        
    def forward(self, x):
        return self.classifier(x)
    
classifier = NeuralNetClassifier(
    ClassifierModule(x_dim= train_x.shape[1], y_dim = train_y.shape[0]),
    max_epochs=10,
    lr=0.1,
    device='cuda' if torch.cuda.is_available() else 'cpu',
    optimizer=torch.optim.Adam,
    criterion=nn.CrossEntropyLoss,
    
)

# train the classifier
classifier.fit(train_x, train_y)

# evaluate the classifier
from sklearn.metrics import accuracy_score

def evaluate_classifier(classifier, x, y):
    y_pred = classifier.predict(x)
    return accuracy_score(y, y_pred)

print(train_x.shape, train_y.shape)
print(dev_x.shape, dev_y.shape)
print(test_x.shape, test_y.shape)

train_acc = evaluate_classifier(classifier, train_x, train_y)
dev_acc = evaluate_classifier(classifier, dev_x, dev_y)
test_acc = evaluate_classifier(classifier, test_x, test_y)

print(f"Train acc.: {train_acc:.4f}\nDev acc.: {dev_acc:.4f}\nTest acc.: {test_acc:.4f}")




OutOfMemoryError: CUDA out of memory. Tried to allocate 600.00 MiB (GPU 0; 4.00 GiB total capacity; 3.05 GiB already allocated; 0 bytes free; 3.05 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

# Trees

For our gold labels, we need to recover the node distances from our parse tree. For this we will use the functionality provided by `ete3`, that allows us to compute that directly. I have provided code that transforms a `TokenTree` to a `Tree` in `ete3` format.

In [12]:
# In case you want to transform your conllu tree to an nltk.Tree, for better visualisation

def rec_tokentree_to_nltk(tokentree):
    token = tokentree.token["form"]
    tree_str = f"({token} {' '.join(rec_tokentree_to_nltk(t) for t in tokentree.children)})"

    return tree_str


def tokentree_to_nltk(tokentree):
    from nltk import Tree as NLTKTree

    tree_str = rec_tokentree_to_nltk(tokentree)

    return NLTKTree.fromstring(tree_str)

In [13]:
# !pip install ete3
from ete3 import Tree as EteTree


class FancyTree(EteTree):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, format=1, **kwargs)
        
    def __str__(self):
        return self.get_ascii(show_internal=True)
    
    def __repr__(self):
        return str(self)


def rec_tokentree_to_ete(tokentree):
    idx = str(tokentree.token["id"])
    children = tokentree.children
    if children:
        return f"({','.join(rec_tokentree_to_ete(t) for t in children)}){idx}"
    else:
        return idx
    
def tokentree_to_ete(tokentree):
    newick_str = rec_tokentree_to_ete(tokentree)

    return FancyTree(f"{newick_str};")

In [14]:
# Let's check if it works!
# We can read in a corpus using the code that was already provided, and convert it to an ete3 Tree.

def parse_corpus(filename):
    from conllu import parse_incr

    data_file = open(filename, encoding="utf-8")

    ud_parses = list(parse_incr(data_file))
    
    return ud_parses

corpus = parse_corpus('data/sample/en_ewt-ud-train.conllu')
item = corpus[0]
tokentree = item.to_tree()
ete3_tree = tokentree_to_ete(tokentree)
print(ete3_tree)


   /-2
  |
  |--3
  |
  |--4
  |
  |   /6 /-5
  |  |
  |  |   /-9
  |  |  |
  |  |  |--10
  |  |  |
  |  |  |--11
  |  |-8|
  |  |  |--12
  |-7|  |
  |  |  |--13
  |  |  |
  |  |   \15/-14
-1|  |
  |  |   /-16
  |  |  |
  |  |  |--17
  |  |  |
  |   \18   /-19
  |     |  |
  |     |  |--20
  |     |  |
  |     |  |-23/-22
  |      \21
  |        |--24
  |        |
  |        |   /-25
  |        |  |
  |         \28--26
  |           |
  |            \-27
  |
   \-29


As you can see we label a token by its token id (converted to a string). Based on these id's we are going to retrieve the node distances.

To create the true distances of a parse tree in our treebank, we are going to use the `.get_distance` method that is provided by `ete3`: http://etetoolkit.org/docs/latest/tutorial/tutorial_trees.html#working-with-branch-distances

We will store all these distances in a `torch.Tensor`.

Please fill in the gap in the following method. I recommend you to have a good look at Hewitt's blog post  about these node distances.

In [51]:
def create_gold_distances(corpus):
    all_distances = []

    for item in (corpus):
        tokentree = item.to_tree()
        ete_tree = tokentree_to_ete(tokentree)

        sen_len = len(ete_tree.search_nodes())
        distances = torch.zeros((sen_len, sen_len))

        for node1 in ete_tree.traverse():
            for node2 in ete_tree.traverse():
                if node1 != node2:
                    distances[int(node1.name)-1, int(node2.name)-1] = node1.get_distance(node2)

        all_distances.append(distances)

    return all_distances

create_gold_distances(corpus[5:7])[1]

tensor([[0., 1., 3., 3., 3., 2., 4., 3., 4., 4., 4., 3., 2.],
        [1., 0., 2., 2., 2., 1., 3., 2., 3., 3., 3., 2., 1.],
        [3., 2., 0., 2., 2., 1., 3., 2., 3., 3., 3., 2., 3.],
        [3., 2., 2., 0., 2., 1., 3., 2., 3., 3., 3., 2., 3.],
        [3., 2., 2., 2., 0., 1., 3., 2., 3., 3., 3., 2., 3.],
        [2., 1., 1., 1., 1., 0., 2., 1., 2., 2., 2., 1., 2.],
        [4., 3., 3., 3., 3., 2., 0., 1., 4., 4., 4., 3., 4.],
        [3., 2., 2., 2., 2., 1., 1., 0., 3., 3., 3., 2., 3.],
        [4., 3., 3., 3., 3., 2., 4., 3., 0., 2., 2., 1., 4.],
        [4., 3., 3., 3., 3., 2., 4., 3., 2., 0., 2., 1., 4.],
        [4., 3., 3., 3., 3., 2., 4., 3., 2., 2., 0., 1., 4.],
        [3., 2., 2., 2., 2., 1., 3., 2., 1., 1., 1., 0., 3.],
        [2., 1., 3., 3., 3., 2., 4., 3., 4., 4., 4., 3., 0.]])

The next step is now to do the previous step the other way around. After all, we are mainly interested in predicting the node distances of a sentence, in order to recreate the corresponding parse tree.

Hewitt et al. reconstruct a parse tree based on a _minimum spanning tree_ (MST, https://en.wikipedia.org/wiki/Minimum_spanning_tree). Fortunately for us, we can simply import a method from `scipy` that retrieves this MST.

In [16]:
from scipy.sparse.csgraph import minimum_spanning_tree
import torch


def create_mst(distances):
    distances = torch.triu(distances).detach().numpy()
    
    mst = minimum_spanning_tree(distances).toarray()
    mst[mst>0] = 1.
    
    return mst

Let's have a look at what this looks like, by looking at a relatively short sentence in the sample corpus.

If your addition to the `create_gold_distances` method has been correct, you should be able to run the following snippet. This then shows you the original parse tree, the distances between the nodes, and the MST that is retrieved from these distances. Can you spot the edges in the MST matrix that correspond to the edges in the parse tree?

In [17]:
item = corpus[5]
tokentree = item.to_tree()
ete3_tree = tokentree_to_ete(tokentree)
print(ete3_tree, '\n')

gold_distance = create_gold_distances(corpus[5:6])[0]
print(gold_distance, '\n')

mst = create_mst(gold_distance)
print(mst)


   /2 /-1
  |
  |--3
  |
  |--4
  |
  |   /-6
  |  |
-5|  |--7
  |-8|
  |  |   /-9
  |  |  |
  |   \12--10
  |     |
  |      \-11
  |
   \-13 

tensor([[0., 1., 3., 3., 2., 4., 4., 3., 5., 5., 5., 4., 3.],
        [1., 0., 2., 2., 1., 3., 3., 2., 4., 4., 4., 3., 2.],
        [3., 2., 0., 2., 1., 3., 3., 2., 4., 4., 4., 3., 2.],
        [3., 2., 2., 0., 1., 3., 3., 2., 4., 4., 4., 3., 2.],
        [2., 1., 1., 1., 0., 2., 2., 1., 3., 3., 3., 2., 1.],
        [4., 3., 3., 3., 2., 0., 2., 1., 3., 3., 3., 2., 3.],
        [4., 3., 3., 3., 2., 2., 0., 1., 3., 3., 3., 2., 3.],
        [3., 2., 2., 2., 1., 1., 1., 0., 2., 2., 2., 1., 2.],
        [5., 4., 4., 4., 3., 3., 3., 2., 0., 2., 2., 1., 4.],
        [5., 4., 4., 4., 3., 3., 3., 2., 2., 0., 2., 1., 4.],
        [5., 4., 4., 4., 3., 3., 3., 2., 2., 2., 0., 1., 4.],
        [4., 3., 3., 3., 2., 2., 2., 1., 1., 1., 1., 0., 3.],
        [3., 2., 2., 2., 1., 3., 3., 2., 4., 4., 4., 3., 0.]]) 

[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0

Now that we are able to map edge distances back to parse trees, we can create code for our quantitative evaluation. For this we will use the Undirected Unlabeled Attachment Score (UUAS), which is expressed as:

$$\frac{\text{number of predicted edges that are an edge in the gold parse tree}}{\text{number of edges in the gold parse tree}}$$

To do this, we will need to obtain all the edges from our MST matrix. Note that, since we are using undirected trees, that an edge can be expressed in 2 ways: an edge between node $i$ and node $j$ is denoted by both `mst[i,j] = 1`, or `mst[j,i] = 1`.

You will write code that computes the UUAS score for a matrix of predicted distances, and the corresponding gold distances. I recommend you to split this up into 2 methods: 1 that retrieves the edges that are present in an MST matrix, and one general method that computes the UUAS score.

In [18]:
import numpy as np
def edges(mst):
    
    edges = np.nonzero(mst)
    edges = list(zip(edges[0], edges[1]))
    edges = set(map(lambda x: tuple(sorted(x)), edges))
    return edges
    '''
    edges = set()

    # Your code for retrieving the edges from the MST matrix

    for i in range(mst.shape[0]):
        for j in range(mst.shape[1]):
            if mst[i,j] == 1:
                edges.add((i,j))
    '''
    return edges


# print(edges(mst))

def calc_uuas(pred_distances, gold_distances):
    uuas = 0
    length = 0
    for i in range(len(gold_distances)):
        l = max(torch.nonzero(gold_distances[i]!=-1, as_tuple=True)[0]) +1
        pred_mst = create_mst(pred_distances[i][:l,:l])
        gold_mst = create_mst(gold_distances[i][:l,:l])
        pred_edges = edges(pred_mst)
        gold_edges = edges(gold_mst)
        try:
            uuas += len(pred_edges.intersection(gold_edges)) / len(gold_edges)    
            length += 1
        except:
            pass
    return uuas / length
    
    '''
    uuas = None
    
    # Your code for computing the UUAS score
    uuas = []
    for pred_distance, gold_distance in zip(pred_distances, gold_distances):

        pred_mst = create_mst(pred_distance)
        gold_mst = create_mst(gold_distance)
        pred_edges = edges(pred_mst)
        gold_edges = edges(gold_mst)

        uua = len(pred_edges.intersection(gold_edges)) / len(gold_edges)
        uuas.append(uua)

    
    return uuas
    '''



# Structural Probes

We now have everything in place to start doing the actual exciting stuff: training our structural probe!
    
To make life easier for you, we will simply take the `torch` code for this probe from John Hewitt's repository. This allows you to focus on the training regime from now on.

In [63]:
import torch.nn as nn
import torch


class StructuralProbe(nn.Module):
    """ Computes squared L2 distance after projection by a matrix.
    For a batch of sentences, computes all n^2 pairs of distances
    for each sentence in the batch.
    """
    def __init__(self, model_dim, rank, device="cpu"):
        super().__init__()
        self.probe_rank = rank
        self.model_dim = model_dim
        
        self.proj = nn.Parameter(data = torch.zeros(self.model_dim, self.probe_rank))
        
        nn.init.uniform_(self.proj, -0.05, 0.05)
        self.to(device)

    def forward(self, batch):
        """ Computes all n^2 pairs of distances after projection
        for each sentence in a batch.
        Note that due to padding, some distances will be non-zero for pads.
        Computes (B(h_i-h_j))^T(B(h_i-h_j)) for all i,j
        Args:
          batch: a batch of word representations of the shape
            (batch_size, max_seq_len, representation_dim)
        Returns:
          A tensor of distances of shape (batch_size, max_seq_len, max_seq_len)
        """
        transformed = torch.matmul(batch, self.proj)
        
        batchlen, seqlen, rank = transformed.size()
        
        transformed = transformed.unsqueeze(2)
        transformed = transformed.expand(-1, -1, seqlen, -1)
        transposed = transformed.transpose(1,2)
        
        diffs = transformed - transposed
        
        squared_diffs = diffs.pow(2)
        squared_distances = torch.sum(squared_diffs, -1)

        return squared_distances

    
class L1DistanceLoss(nn.Module):
    """Custom L1 loss for distance matrices."""
    def __init__(self):
        super().__init__()

    def forward(self, predictions, label_batch, length_batch):
        """ Computes L1 loss on distance matrices.
        Ignores all entries where label_batch=-1
        Normalizes first within sentences (by dividing by the square of the sentence length)
        and then across the batch.
        Args:
          predictions: A pytorch batch of predicted distances
          label_batch: A pytorch batch of true distances
          length_batch: A pytorch batch of sentence lengths
        Returns:
          A tuple of:
            batch_loss: average loss in the batch
            total_sents: number of sentences in the batch
        """
        labels_1s = (label_batch != -1).float()
        predictions_masked = predictions * labels_1s
        labels_masked = label_batch * labels_1s
        total_sents = torch.sum((length_batch != 0)).float()
        squared_lengths = length_batch.pow(2).float()

        if total_sents > 0:
            loss_per_sent = torch.sum(torch.abs(predictions_masked - labels_masked), dim=(1,2))
            normalized_loss_per_sent = loss_per_sent / squared_lengths
            batch_loss = torch.sum(normalized_loss_per_sent) / total_sents
        
        else:
            batch_loss = torch.tensor(0.0)
        
        return batch_loss, total_sents


I have provided a rough outline for the training regime that you can use. Note that the hyper parameters that I provide here only serve as an indication, but should be (briefly) explored by yourself.

As can be seen in Hewitt's code above, there exists functionality in the probe to deal with batched input. It is up to you to use that: a (less efficient) method can still incorporate batches by doing multiple forward passes for a batch and computing the backward pass only once for the summed losses of all these forward passes. (_I know, this is not the way to go, but in the interest of time that is allowed ;-), the purpose of the assignment is writing a good paper after all_).

In [64]:
from torch import optim

'''
Similar to the `create_data` method of the previous notebook, I recommend you to use a method 
that initialises all the data of a corpus. Note that for your embeddings you can use the 
`fetch_sen_reps` method again. However, for the POS probe you concatenated all these representations into 
1 big tensor of shape (num_tokens_in_corpus, model_dim). 

The StructuralProbe expects its input to contain all the representations of 1 sentence, so I recommend you
to update your `fetch_sen_reps` method in a way that it is easy to retrieve all the representations that 
correspond to a single sentence.
''' 

def init_corpus(path, model_type='gpt2', concat=False, cutoff=None):
    """ Initialises the data of a corpus.
    
    Parameters
    ----------
    path : str
        Path to corpus location
    concat : bool, optional
        Optional toggle to concatenate all the tensors
        returned by `fetch_sen_reps`.
    cutoff : int, optional
        Optional integer to "cutoff" the data in the corpus.
        This allows only a subset to be used, alleviating 
        memory usage.
    """
    corpus = parse_corpus(path)[:cutoff]
    if model_type == 'gpt2':
        curr_model = model
        embs = fetch_sen_reps(corpus, curr_model, tokenizer, concat=concat)
    elif model_type == 'lstm':
        curr_model = lstm
        embs = fetch_sen_reps(corpus, curr_model, vocab, concat=concat)
    else:
        raise NotImplementedError

    gold_distances = create_gold_distances(corpus)
    
    return list(zip(embs, gold_distances))


def custom_collate_fn(batch):
    max_length = max([len(x[0]) for x in batch])
    out_labels = torch.full((len(batch), max_length, max_length), -1)
    out_lengths = torch.zeros(len(batch))
    for i, x in enumerate(batch):
        out_labels[i, :x[1].shape[0], :x[1].shape[1]] = x[1]
        out_lengths[i] = x[1].shape[0]
        if len(x[0].shape) == 1:
            batch[i] = (x[0].unsqueeze(0), x[1])
    return torch.nn.utils.rnn.pad_sequence(list(map(lambda x: x[0].detach(), batch)), batch_first = True, padding_value=-1), out_labels, out_lengths



def get_data(root="data", model_type='gpt2', lang='en', batch_size=32):
    if lang == 'en':
        train_path = root + '/en_ewt-ud-train.conllu' 
        val_path = root + '/en_ewt-ud-dev.conllu'
        test_path = root + '/en_ewt-ud-test.conllu'
    else:
        raise NotImplementedError
    
    train_data = init_corpus(train_path, model_type)
    val_data = init_corpus(val_path, model_type)
    test_data = init_corpus(test_path, model_type)

    print(test_path)

    train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, num_workers=0, shuffle=True,
        collate_fn=custom_collate_fn,) 
    val_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, num_workers=0, shuffle=False,
        collate_fn=custom_collate_fn)
    test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, num_workers=0, shuffle=False,
        collate_fn=custom_collate_fn)
    return train_loader, val_loader, test_loader



# I recommend you to write a method that can evaluate the UUAS & loss score for the dev (& test) corpus.
# Feel free to alter the signature of this method.
def evaluate_probe(probe, dataloader):
    # YOUR CODE HERE
    probe.eval()
    loss_score = 0
    uuas_score = 0
    loss_function =  L1DistanceLoss()
    
    with torch.no_grad():
        for i, eval_batch in enumerate(dataloader):
            x = eval_batch[0]
            gold_distances = eval_batch[1]
            lengths = eval_batch[2]

            preds = probe(x)

            loss_score += loss_function(preds, gold_distances, lengths)[0]
            uuas_score += calc_uuas(preds, gold_distances)

            
    loss_score /= len(dataloader)
    uuas_score /= len(dataloader)

        
    return loss_score, uuas_score


# Feel free to alter the signature of this method.
def train(root='data/', model_type='gpt2'):
    if model_type == "gpt2":
        emb_dim = 768
    elif model_type == "lstm":
        emb_dim = 650
    rank = 64
    lr = 10e-4
    batch_size = 16
    epochs = 5

    probe = StructuralProbe(emb_dim, rank)
    optimizer = optim.Adam(probe.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5,patience=1)
    loss_function =  L1DistanceLoss()
    
    train_loader, val_loader, test_loader = get_data(root, model_type=model_type, lang="en", batch_size=batch_size)

    for epoch in range(epochs):


        for train_batch in train_loader:
            optimizer.zero_grad()

            # YOUR CODE FOR DOING A PROBE FORWARD PASS
            
            x = train_batch[0]
            gold_distances = train_batch[1]
            lengths = train_batch[2]

            preds = probe(x)

            batch_loss, _ = loss_function(preds, gold_distances, lengths)
            
            batch_loss.backward()
            optimizer.step()


        dev_loss, dev_uuas = evaluate_probe(probe, val_loader)
        print("dev_loss = ", dev_loss)

        # Using a scheduler is up to you, and might require some hyper param fine-tuning
        scheduler.step(dev_loss)
    
    test_loss, test_uuas = evaluate_probe(probe, test_loader)
    print("test_loss = ", test_loss)
    print("test_uuas = ", test_uuas)
    return probe


In [67]:
lstm_probe = train(root='data/sample', model_type='lstm')

debug fetch_sen_reps LSTM, concat= False
debug fetch_sen_reps LSTM, concat= False
debug fetch_sen_reps LSTM, concat= False
data/sample/en_ewt-ud-test.conllu
dev_loss =  tensor(1.2814)
dev_loss =  tensor(1.2152)
dev_loss =  tensor(1.1710)
dev_loss =  tensor(1.1014)
dev_loss =  tensor(1.0612)
test_loss =  tensor(1.1250)
test_uuas =  0.3914261172322513


In [68]:
gpt2_probe = train(root='data/sample', model_type='gpt2')

debug fetch_sen_reps GPT2, concat= False
debug fetch_sen_reps GPT2, concat= False
debug fetch_sen_reps GPT2, concat= False
data/sample/en_ewt-ud-test.conllu
dev_loss =  tensor(65.1473)
dev_loss =  tensor(33.4059)
dev_loss =  tensor(21.1845)
dev_loss =  tensor(15.4552)
dev_loss =  tensor(12.0346)
test_loss =  tensor(14.0666)
test_uuas =  0.17753873854184393


## LaTeX trees

For your report you might want to add some of those fancy dependency tree plots like those of Figure 2 in the Structural Probing paper. For that you can use the following code, that outputs the corresponding LaTeX markup.

**N.B.**: for the latex tikz tree the first token in a sentence has index 1 (instead of 0), so take that into account with the predicted and gold edges that you pass to the method.

In [ ]:
def print_tikz(predicted_edges, gold_edges, words):
    """ Turns edge sets on word (nodes) into tikz dependency LaTeX.
    Parameters
    ----------
    predicted_edges : Set[Tuple[int, int]]
        Set (or list) of edge tuples, as predicted by your probe.
    gold_edges : Set[Tuple[int, int]]
        Set (or list) of gold edge tuples, as obtained from the treebank.
    words : List[str]
        List of strings representing the tokens in the sentence.
    """

    string = """\\begin{dependency}[hide label, edge unit distance=.5ex]
    \\begin{deptext}[column sep=0.05cm]
    """

    string += (
        "\\& ".join([x.replace("$", "\$").replace("&", "+") for x in words])
        + " \\\\\n"
    )
    string += "\\end{deptext}" + "\n"
    for i_index, j_index in gold_edges:
        string += "\\depedge[-]{{{}}}{{{}}}{{{}}}\n".format(i_index, j_index, ".")
    for i_index, j_index in predicted_edges:
        string += f"\\depedge[-,edge style={{red!60!}}, edge below]{{{i_index}}}{{{j_index}}}{{.}}\n"
    string += "\\end{dependency}\n"
    print(string)

## Correlation experiments

In [92]:


root='data'
model_type='lstm'
train_loader, val_loader, test_loader  = get_data(root, model_type, lang='en', batch_size=1)

debug fetch_sen_reps LSTM, concat= False


RuntimeError: [enforce fail at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 21197670000 bytes.

In [69]:
st_parses = parse_corpus('data/sample/en_ewt-ud-test.conllu')
for sent in st_parses:
    print(len(sent))

34
38
22
20
22
5
28
36
33
29
19
24
28
17
14
19
19
26
25
23
13
20
31
33
16
25
17
34
25
20
18
25
13
31
40
21
19
18
21
26
14
16
36
24
28
14
19
11
21
23


In [ ]:
# import pearsonr
from scipy.stats import pearsonr
from scipy.stats import spearmanr

def calc_corr(probe, classifier, test_x, test_y, test_loader):

    probe.eval()
    i = 0
    diag_accs = []
    uuas_scores = []

    with torch.no_grad():
        for sample in test_loader:
            probe_x = sample[0]
            gold_distances = sample[1]
            lengths = int(sample[2][0])

            diag_x = test_x[i:i+lengths]
            diag_y = test_y[i:i+lengths]

            
            diag_accs.append(evaluate_classifier(classifier, diag_x, diag_y))
            
            preds = probe(probe_x)
            uuas_scores.append(calc_uuas(preds, gold_distances))
            
            i += lengths
            
    # calculate pearson correlation between diag_accs and uuas_scores
    pearson = pearsonr(diag_accs, uuas_scores)

    #calculate spearman correlation between diag_accs and uuas_scores
    spearman = spearmanr(diag_accs, uuas_scores)

    return pearson, spearman
            

pearson, spearman = calc_corr(lstm_probe, classifier, test_x, test_y, test_loader)

print("Pearson correlation: ", pearson)
print("Spearman correlation: ", spearman)

Pearson correlation:  (0.0184732042091259, 0.8986774892207444)
Spearman correlation:  SpearmanrResult(correlation=-0.010049768062420847, pvalue=0.9447771591339427)
